In [15]:
from configparser import ConfigParser
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))
input = pd.read_csv('artists.csv', header = None)
artist_ids = []
for link in input[0]:
    
    artist_ids.append(link[32:54])

artist_ids
albums = []
album_track_ids = []

for artist_id in artist_ids:
    
    album_ids = []
    artist_uri = 'spotify:artist:' + artist_id

    # Add album ids to album_ids list
    results = sp.artist_albums(artist_uri, album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    for album in albums:
        album_ids.append(album['id'])


for id in album_ids:

    track_ids = []

    results = sp.album_tracks(id)
    tracks = results['items']
    for track in tracks:
        track_ids.append(track['id'])

    album_track_ids.append(track_ids)


assert len(album_track_ids) == len(albums)

len(albums)
columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
headings = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'id', 'duration_ms', 'time_signature','album_title', 'album_artist', 'album position']
data = pd.DataFrame(columns=headings)


# Create DataFrames of features from track ids
i = 0
for album in album_track_ids:

    features = []
    album_position = []
    j = 0
    for track in album:
        results = sp.audio_features(track)
        features.append(results[0])
        album_position.append(j/len(album))
        j += 1
    features_matrix = pd.DataFrame.from_records(features)

    # Remove unneeded columns
    features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)

    feature_lists = []
    for column in features_matrix:
        feature_lists.append(list(features_matrix[column]))

    # Add album title
    title = albums[i]['name']
    feature_lists.append(title)
    # Add artist name
    name = albums[i]['artists'][0]['name']
    feature_lists.append(name)
    # Add album position
    feature_lists.append(album_position)

    # Add album as a row to data df
  
    data.loc[i] = feature_lists
    i += 1

data.drop_duplicates(subset=['album_title', 'album_artist'])
data.to_csv('./data.csv')

In [35]:
# Set dataset

#import matplotlib.pyplot as plt

data = pd.read_csv('data.csv', header = None)

columns_to_remove = ['id',
                    'album_title',]
#                    'album_artist']

data.drop(columns = columns_to_remove, axis = 1, inplace = True)

data

KeyError: "['id' 'album_title'] not found in axis"

In [17]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(danceabilities_test, album_positions_test, sample_weight=None)

ValueError: Expected 2D array, got 1D array instead:
array=['[' '0' '.' ... '6' '2' ']'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.